##### Bibliotecas necessárias e configurações de ambiente

In [1]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, desc, udf
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

# Set variáveis de ambiente
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Cria uma sessão Spark
spark=SparkSession.builder \
    .appName("actdigital") \
    .getOrCreate()

In [3]:
# Dados disponibilizados
data=[
    ("Alice", 34, "Data Scientist"),
    ("Bob", 45, "Data Engineer"),
    ("Cathy", 29, "Data Analyst"),
    ("David", 35, "Data Scientist")
]
columns=["Name", "Age", "Occupation"]

##### Parte 1: Manipulação de Dados

In [6]:
# Crie um DataFrame a partir dos dados fornecidos
df=spark.createDataFrame(data, schema=columns)
df.show()

+-----+---+--------------+
| Name|Age|    Occupation|
+-----+---+--------------+
|Alice| 34|Data Scientist|
|  Bob| 45| Data Engineer|
|Cathy| 29|  Data Analyst|
|David| 35|Data Scientist|
+-----+---+--------------+



In [7]:
# Selecione apenas as colunas "Name" e "Age" 
df.select("name", "age").show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
|David| 35|
+-----+---+



In [13]:
# Filtre as linhas onde a "Age" é maior que 30
df.filter("Age > 30").show()

+-----+---+--------------+
| Name|Age|    Occupation|
+-----+---+--------------+
|Alice| 34|Data Scientist|
|  Bob| 45| Data Engineer|
|David| 35|Data Scientist|
+-----+---+--------------+



In [19]:
# Agrupe os dados pelo campo "Occupation" e calcule a média de "Age" para cada grupo
df.groupBy("Occupation").agg(
    avg("Age")
    ).show()

+--------------+--------+
|    Occupation|avg(Age)|
+--------------+--------+
|Data Scientist|    34.5|
| Data Engineer|    45.0|
|  Data Analyst|    29.0|
+--------------+--------+



In [23]:
# Ordene o DataFrame resultante da questão anterior pela média de "Age" em ordem decrescente
df.groupBy("Occupation").agg(
    avg("Age")).orderBy(desc(
    "avg(Age)")).show()

+--------------+--------+
|    Occupation|avg(Age)|
+--------------+--------+
| Data Engineer|    45.0|
|Data Scientist|    34.5|
|  Data Analyst|    29.0|
+--------------+--------+



##### Parte 2: Funções Avançadas

In [28]:
# Crie uma função em Python que converte idades para categorias
# e aplique esta função ao DataFrame usando uma UDF

# Função python
def age_category(age):
    if age < 30:
        return "Jovem"
    elif 30 < age < 40:
        return "Adulto"
    else:
        return "Senior"

# Converter para UDF
age_category_udf=udf(age_category, StringType())

# Criar coluna com categorias
df=df.withColumn("AgeCategory", age_category_udf(df["Age"]))
df.show()

+-----+---+--------------+-----------+
| Name|Age|    Occupation|AgeCategory|
+-----+---+--------------+-----------+
|Alice| 34|Data Scientist|     Adulto|
|  Bob| 45| Data Engineer|     Senior|
|Cathy| 29|  Data Analyst|      Jovem|
|David| 35|Data Scientist|     Adulto|
+-----+---+--------------+-----------+



In [31]:
# Adicione uma coluna que mostre a diferença de idade entre
# cada indivíduo e a média de idade do seu "Occupation"

# Especificar condicional da janela
windowSpec=Window.partitionBy("Occupation")

# Calcular a idade média da ocupação
df=df.withColumn("AvgOccupationAge", avg(
    "Age").over(windowSpec))

# Diferença em relação à média da ocupação
df=df.withColumn("AgeDifference", col(
    "Age") - col("AvgOccupationAge"))
df.show()

+-----+---+--------------+-----------+----------------+-------------+
| Name|Age|    Occupation|AgeCategory|AvgOccupationAge|AgeDifference|
+-----+---+--------------+-----------+----------------+-------------+
|Cathy| 29|  Data Analyst|      Jovem|            29.0|          0.0|
|  Bob| 45| Data Engineer|     Senior|            45.0|          0.0|
|Alice| 34|Data Scientist|     Adulto|            34.5|         -0.5|
|David| 35|Data Scientist|     Adulto|            34.5|          0.5|
+-----+---+--------------+-----------+----------------+-------------+



##### Parte 3: Performance e Otimização

##### Parte 4: Integração com Outras Tecnologias

##### Parte 5: Problema de Caso